In [ ]:
!pip install torch==2.3.0 torchvision torchaudio
!pip install intel_extension_for_pytorch

  Using cached intel_extension_for_pytorch-2.3.0-cp310-cp310-manylinux2014_x86_64.whl (98.3 MB)


In [ ]:
!pip install --upgrade jax jaxlib
!pip install --upgrade diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.3 MB/s eta 0:00:00


In [ ]:
import os
import warnings

warnings.filterwarnings("ignore")

import random
import requests
import torch
import intel_extension_for_pytorch as ipex
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionImg2ImgPipeline
#from diffusers import StableDiffusionPipeline

import torch.nn as nn
import time
from typing import List, Dict, Tuple


class Img2ImgModel:


    def __init__(
        self,
        model_id_or_path: str,
        device: str = "cuda",
        torch_dtype: torch.dtype = torch.float16,
        optimize: bool = True,
    ) -> None:

        self.device = device
        self.pipeline = self._load_pipeline(model_id_or_path, torch_dtype)
        if optimize:
            start_time = time.time()
            print("Optimizing the model...")
            self.optimize_pipeline()
            print(
                "Optimization completed in {:.2f} seconds.".format(
                    time.time() - start_time
                )
            )

    def _load_pipeline(self, model_id_or_path: str, torch_dtype: torch.dtype) -> StableDiffusionImg2ImgPipeline:
          print("Loading the model...")
          pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
              model_id_or_path, torch_dtype=torch_dtype
          )
          pipeline = pipeline.to(self.device)
          print("Model loaded.")
          return pipeline


    def _optimize_pipeline(self, pipeline: StableDiffusionImg2ImgPipeline) -> StableDiffusionImg2ImgPipeline:
          if isinstance(pipeline, nn.Module):
              pipeline = ipex.optimize(pipeline.eval(), dtype=pipeline.text_encoder.dtype, inplace=True)
          return pipeline


    def optimize_pipeline(self) -> None:
          """
          Optimize the pipeline of the model.
          """
          print("Optimizing the model...")
          start_time = time.time()
          self.pipeline = self._optimize_pipeline(self.pipeline)
          print("Optimization completed in {:.2f} seconds.".format(time.time() - start_time))

    def get_image_from_url(self, url: str, path: str) -> Image.Image:
        """
        Get an image from a URL or from a local path if it exists.

        Args:
            url (str): The URL of the image.
            path (str): The local path of the image.

        Returns:
            Image.Image: The loaded image.
        """
        if os.path.exists(path):
            img = Image.open(path).convert("RGB")
        else:
            response = requests.get(url)
            if response.status_code != 200:
                raise Exception(
                    f"Failed to download image. Status code: {response.status_code}"
                )
            if not response.headers["content-type"].startswith("image"):
                raise Exception(
                    f"URL does not point to an image. Content type: {response.headers['content-type']}"
                )
            img = Image.open(BytesIO(response.content)).convert("RGB")
            img.save(path)
        img = img.resize((768, 512))
        return img

    @staticmethod
    def random_sublist(lst):
        sublist = []
        for _ in range(random.randint(1, len(lst))):
            item = random.choice(lst)
            sublist.append(item)
        return sublist

    def generate_images(
        self,
        prompt: str,
        image_url: str,
        class_name: str,
        seed_image_identifier: str,
        variations: List[str],
        num_images: int = 5,
        strength: float = 0.75,
        guidance_scale: float = 7.5,
        save_path: str = "output",
        seed_path: str = "intput",
    ) -> List[Image.Image]:
        
        input_image_path = f"{seed_path}/{seed_image_identifier}.png"
        init_image = self.get_image_from_url(image_url, input_image_path)
        images = []
        for i in range(num_images):
            variation = variations[i % len(variations)]
            final_prompt = f"{prompt} {variation}"
            image = self.pipeline(
                prompt=final_prompt,
                image=init_image,
                strength=strength,
                guidance_scale=guidance_scale,
            ).images
            output_image_path = os.path.join(
                save_path,
                f"{seed_image_identifier}_{'_'.join(variation.split())}_{i}.png",
            )
            image[0].save(output_image_path)
            images.append(image)
        return images


if __name__ == "__main__":
    model_id = "runwayml/stable-diffusion-v1-5" #####
    base_prompt = (
        "A close image to this original satellite image with slight change in location"
    )
    fire_variations = [
        "early morning with a wild fire",
        "late afternoon",
        "mid-day",
        "night with wild fire",
        "smoky conditions",
        "visible fire lines",
    ]
    no_fire_variations = [
        "early morning with clear skies",
        "no signs of fire",
        "night",
        "late afternoon with clear skies",
        "mid-day with clear skies",
        "with dense vegetation",
        "with sparse vegetation",
    ]

    image_urls = {
        "fire": [
            "https://github.com/intelsoftware/ForestFirePrediction/blob/main/data/real_USGS_NAIP/train/Fire/m_3912105_sw_10_h_20160713.png?raw=true",
            "https://github.com/intelsoftware/ForestFirePrediction/blob/main/data/real_USGS_NAIP/train/Fire/m_3912113_sw_10_h_20160713.png?raw=true",
            "https://github.com/intelsoftware/ForestFirePrediction/blob/main/data/real_USGS_NAIP/train/Fire/m_3912114_se_10_h_20160806.png?raw=true",
            "https://github.com/intelsoftware/ForestFirePrediction/blob/main/data/real_USGS_NAIP/train/Fire/m_3912120_ne_10_h_20160713.png?raw=true",
            "https://github.com/intelsoftware/ForestFirePrediction/blob/main/data/real_USGS_NAIP/train/Fire/m_4012355_se_10_h_20160713.png?raw=true",
        ],
        "no_fire": [
            "https://github.com/intelsoftware/ForestFirePrediction/blob/main/data/real_USGS_NAIP/train/NoFire/m_3912045_ne_10_h_20160712.png?raw=true",
            "https://github.com/intelsoftware/ForestFirePrediction/blob/main/data/real_USGS_NAIP/train/NoFire/m_3912057_sw_10_h_20160711.png?raw=true",
            "https://github.com/intelsoftware/ForestFirePrediction/blob/main/data/real_USGS_NAIP/train/NoFire/m_3912142_sw_10_h_20160711.png?raw=true",
            "https://github.com/intelsoftware/ForestFirePrediction/blob/main/data/real_USGS_NAIP/train/NoFire/m_3912343_se_10_h_20160529.png?raw=true",
            "https://github.com/intelsoftware/ForestFirePrediction/blob/main/data/real_USGS_NAIP/train/NoFire/m_4012241_se_10_h_20160712.png?raw=true",
        ],
    }

    model = Img2ImgModel(model_id, device="cuda")
    num_images = 5
    gen_img_count = 0

    try:
        start_time = time.time()
        for class_name, urls in image_urls.items():
            for url in urls:
                seed_image_identifier = os.path.basename(url).split(".")[0]
                input_dir = f"./input/{class_name}"
                output_dir = f"./output/{class_name}"
                os.makedirs(input_dir, exist_ok=True)
                os.makedirs(output_dir, exist_ok=True)
                variations = (
                    fire_variations if class_name == "fire" else no_fire_variations
                )
                model.generate_images(
                    base_prompt,
                    url,
                    class_name,
                    seed_image_identifier,
                    variations=variations,
                    save_path=output_dir,
                    seed_path=input_dir,
                    num_images=num_images,
                )
                gen_img_count += num_images
    except KeyboardInterrupt:
        print("\nUser interrupted image generation...")
    finally:
        print(
            f"Complete generating {gen_img_count} images in {'/'.join(output_dir.split('/')[:-1])} in {time.time() - start_time:.2f} seconds."
        )

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading the model...


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

Complete generating 50 images in ./output in 221.94 seconds.


In [ ]:
import shutil

def compress_folder(folder_path, output_path, archive_format='zip'):

    shutil.make_archive(output_path, archive_format, folder_path)


folder_to_compress = "/content/output"
output_file_path = "stable_diffussion_output"
compress_folder(folder_to_compress, output_file_path)

In [ ]:
def unpack_archive(archive_path, extract_path):

    shutil.unpack_archive(archive_path, extract_path)


archive_to_extract = "/path/to/your/compressed_file.zip"
destination_path = "/path/to/extract/folder"
unpack_archive(archive_to_extract, destination_path)